# Project 4

Write a Python function/code to find the modular inverse of a given matrix (nxn) in mod m. A matrix and m are supposed to be provided by user.

The aim of this Assignment is to get familiar with Python programming and to improve algorithm development skills.
Modular arithmetic plays crucial role in Cryptography. Modular inverse of a matrix is one of the key and compulsory steps in the Hill Cipher.

## Modular Arithmetic

For a Mod b, divide a by b and take the remainder.
- 14 ÷ 10 = 1 R 4
- 14 Mod 10 = 4
- 24 Mod 10 = 4

## Modulus Theorem

![](https://raw.githubusercontent.com/mi-ng/i116-assignments/master/img/4_1.png)

![](https://raw.githubusercontent.com/mi-ng/i116-assignments/master/img/4_2.png)

## Modular Inverses
- Inverse of 2 is 1⁄2 (2 · 1⁄2 = 1)
- MatrixInverse: AA-1=I
- Modular Inverse of a for Mod m: (a · a-1) Mod m = 1
- For Modular Inverses, a and m must NOT have any prime factors in common

- Example: Modular Inverses of Mod 26
  - Find the Modular Inverse of 9 for Mod 26 9 · 3 = 27
  - 27 Mod 26 = 1
  - 3 is the Modular Inverse of 9 Mod 26

## Modularly Inverse Matrices
- Calculate determinant of first matrix A, det A
- Make sure that det A has a modular inverse for Mod 26
- Calculate the adjugate of A, adj A
- Multiply adj A by modular inverse of det A
- Calculate Mod 26 of the result to get B
- Use A to encrypt, B to decrypt







In [0]:
import math
import numpy as np
import matplotlib.pyplot as plt


In [0]:
class Matrix2D:
  def __init__(self, arr):
    self.a = arr[0]
    self.b = arr[1]
    self.c = arr[2]
    self.d = arr[3]
  def to_string(self):
    return str(self.a) + ", " + str(self.b) + ", " + str(self.c) + ", "  + str(self.d)
  def to_array(self):
    return [self.a, self.b, self.c, self.d]

In [0]:
latin_alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"

def get_code(c, char_map):
  for i in range(len(char_map)):
    if (c == char_map[i]):
      return i
  print("code is not found", c)
  return -1

def get_char(i, char_map):
  return char_map[i]

def print_matrix(A):
  print("");

def det(m):
  return (m.a*m.d) - (m.c*m.b)

def dot(A, b1):
  b2_a = A.a*b1[0] + A.b*b1[1]
  b2_b = A.c*b1[0] + A.d*b1[1]
  return [b2_a, b2_b]

def adj(m):
  a =  m.d
  b = -m.b
  c = -m.c
  d =  m.a
  return Matrix2D([a, b, c, d])

def find_modular_inverse_mod_26(a):
  result = 0;
  i = 1
  while (result == 0):
    if ((i*a)%26 == 1):
      result = i
    i += 1
  return result

def multiply_matrix_number(A, n):
  a = A.a * n
  b = A.b * n
  c = A.c * n
  d = A.d * n
  return Matrix2D([a, b, c, d])

def modular_matrix_number(A, n):
  a = A.a % n
  b = A.b % n
  c = A.c % n
  d = A.d % n
  return Matrix2D([a, b, c, d])

def generate_decrypt_key(K):
  det_K = det(K)
  mod_inverse_det_K = find_modular_inverse_mod_26(det_K)
  K_T = adj(K)
  DK = modular_matrix_number(multiply_matrix_number(K_T, mod_inverse_det_K), 26)

  return DK

def hill_cipher_encrypt(K, text):
  n = int(math.log2(len(K.to_array())))
  tokens = []

  # Tokenize text into pieces that are fitable with size of K
  j = 0
  token = []
  for t in text:
    token.append(t)
    if (j != 0 and (n-1)%j == 0):
      tokens.append(token)
      token = []
      j = 0
    else:
      j += 1
      
  # Encrypt tokens
  ets = []
  for t in tokens:
    ct1 = get_code(t[0], latin_alphabet)
    ct2 = get_code(t[1], latin_alphabet)
    tmp = dot(K, [ct1, ct2])
    et = [tmp[0]%26, tmp[1]%26]
    ets.append(et)

  # flatten
  ets2 = []
  for et in ets:
    for c in et:
      ets2.append(get_char(c, latin_alphabet))
  
  return "".join(ets2)


def hill_cipher_decrypt(DK, encrypt_text):
  n = int(math.log2(len(DK.to_array())))
  tokens = []

  # Tokenize text into pieces that are fitable with size of K
  j = 0
  token = []
  for t in encrypt_text:
    token.append(t)
    if (j != 0 and (n-1)%j == 0):
      tokens.append(token)
      token = []
      j = 0
    else:
      j += 1
      
  # Decrypt tokens
  ts = []
  for t in tokens:
    ct1 = get_code(t[0], latin_alphabet)
    ct2 = get_code(t[1], latin_alphabet)
    tmp = dot(DK, [ct1, ct2])
    t = [tmp[0]%26, tmp[1]%26]
    ts.append(t)

  # flatten
  ts2 = []
  for t in ts:
    for c in t:
      ts2.append(get_char(c, latin_alphabet))
  
  return "".join(ts2)

In [0]:
# Test matrix functions
A = Matrix2D([2, 1, 3, 4])
B = generate_decrypt_key(A)

det_A = det(A)

assert det_A == 5
assert find_modular_inverse_mod_26(5) == 21
assert B.to_array() == [6, 5, 15, 16]
assert get_code("H", latin_alphabet) == 7
assert get_code("E", latin_alphabet) == 4

# Test Hill cipher algorithm
plain_text = "HELP"

encrypted_text = hill_cipher_encrypt(A, plain_text)
decrypted_text = hill_cipher_decrypt(B, encrypted_text)

assert decrypted_text == plain_text

print("key:            ", A.to_string())
print("inverse_key:    ", B.to_string())
print("plain_text:     ", plain_text)
print("encrypted_text: ", encrypted_text)
print("decrypted_text: ", decrypted_text)

print("-----------------------")
print("All test are passed! 🎉")


key:             2, 1, 3, 4
inverse_key:     6, 5, 15, 16
plain_text:      HELP
encrypted_text:  SLLP
decrypted_text:  HELP
-----------------------
All test are passed! 🎉
